# MP 6: Self-supervised Pick and Place

**Due date**: May 4, 2022 at 9:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install. 

There is a small bug discovered in Klampt that will require you to patch your local install.  Run
- `git clone https://github.com/krishauser/Klampt`
- `cd Klampt/Python`
- `python patch_a_pip_install.py`

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Submit this file, all of your .py files, 10 examples (from Problem 1), and any models used in your final pick-and-place planner on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu). 
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Context

This assignment brings together bits and pieces from all that we've learned over the semester, using  the Wlkata Mirobot with a vacuum gripper and an overhead RGBD camera.  The goal is to be able to clear all the blocks to a pile off to the side. If you run `python simulated_robot.py` you will be able to control the robot manually by right-dragging with the mouse, and pressing 'v' to toggle the simulated vacuum gripper.

The main class you will use to interact with the robot is `SimulatedMirobotWithVacuumAndSensor`, found in `simulated_robot.py`. If you are successful, we will be able to replace this with the real robot using the same API, and we can test your code on the real robot!

The underlying arm controller respects the [Klampt Control package](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-Control.html) API.  Your code will be operated in synchronous mode, which steps your application code in alignment with the underlying controller.

## Problem 1: Pick-and-Place using Omniscient Sensing

In `pick_and_place_omniscient.py` you are given all of the object poses in the scene. 

1.A. Use this information to implement a behavior that picks all of the objects using the vacuum and places them in the target location indicated at the top of the file.

You will want to implement a state machine that waits until each movement is done. Because we know that there are no obstacles, you don't strictly need to perform motion planning.

1.B. In the written responses below, describe what measures you needed to take in your implementation to counteract the positioning errors of this inaccurate robot.

## Problem 2: Self-Supervised Simulation-based Training

In `self_supervised.py`, you will implement a self-supervised method for generating data to predict whether an object is graspable. 

2.A. Implement a behavior that moves the robot out of the way, reads the RGB-D image, attempts a grasp at a certain location (according to your heuristic), tests whether the grasp was successful, and then drops the object.  Afterwards, you will save the images and the grasp data to disk in the `generated_data` folder.

It is up to you whether you wish to regress a grasp location or a grasp score.

2.B. Implement a world reset function that allows you to re-attempt grasps after each attempt.  Use this to generate many examples automatically.

2.C. In the written responses below, describe in the area for written responses the heuristic that you use for generated grasps.  How did you attain more positive grasps than random sampling?

2.D. Save 10 example image pairs in PNG format and 10 example grasp data in whatever format you prefer.  Include these with your submission.



## Problem 3: Pick-and-Place using Images

Finally, in `pick_and_place.py` you are only given images of the scene. You should NOT inspect the world model or simulation to determine where the objects are.

3.A. Use the provided information to implement the same pick and place behavior, only using image information, flow sensor information, and the calibrated camera transform `Tcamera_world`.  (You are free to use whatever method you wish to determine object locations -- you will be tested on similar objects with the same color, just in different locations.)

3.B. In the written responses below, describe the process that you used to predict where to grasp objects.  Be specific about what model you used for learning, and how you convert the learned quantities to 3D coordinates.

3.C. In the written responses below, describe sources of error that influence this pick-and-place method.

## Written responses

### Written response for Problem 1.B

To overcome the inaccurate robot, it would be nice if we can actually know the position of the end-effector given camera image. With the visual feedback, we can correct the inaccurate robot. 

### Written response for Problem 2.B

The heuristic that I am using is based on the depth information contained in the image. I am only sample points that have depth below certain threshold to ensure that the points sampled have more positive grasps. 

### Written response for Problem 3.B

I am using a neural network to predict where to grasp object. The neural network have two hidden fully connected layers with 32 hidden neurons, and one output fully connected layer. The non-linearality used is ReLU for both layers. The input to the neural network is the a pixel location in the image and the output from the neural network is it's corresponding x-y coordinate in the world frame. 

The pixel locations are determined by the following steps. 

1. Thresholding the depth image to get all pixels corresponding to the blocks
2. Use hierarchical clustering to classify the pixels into each block
3. Get the average of pixel location for each cluster to get the center of the block as the input to the neural network

### Written response for Problem 3.C

I think the method is generally pretty accurate. The source of error can come from:

1. If a block is not fully covered in the image, the algorithm that I am using can have a hard time localizing and grasp the block
2. If two blocks are too close to each other, the clustering algorithm may failed to classify pixels from different blocks and causing the grasping algorithm to fail. 